In [1]:
import uuid
import pandas as pd
from pathlib import Path
import requests
import re
import csv
import datetime
import time
from threading import Thread

In [2]:
COMPANIES_FILENAME = 'top_ten.csv'
LOG_FILENAME = 'data_acquisition_log'
PAGESPEED_RESULTS_DIR= 'pagespeed_results'
INDEED_RATINGS_DIR = 'indeed_ratings'
PAGESPEED_BASE_URL = 'https://www.googleapis.com/pagespeedonline/v5/runPagespeed'
with open('PAGESPEED_API_KEY') as key_file:
  PAGESPEED_API_KEY = key_file.read().strip() # bad style? should be constant per run
AUDIT_KEYS = [
  'first-contentful-paint',
  'speed-index',
  'interactive',
  'first-meaningful-paint',
  'first-cpu-idle',
  'estimated-input-latency'
]
METADATA_KEYS = ['url']
RESULT_KEYS = METADATA_KEYS + AUDIT_KEYS
TRUNCATE_COMPANIES = None

In [3]:
def last_modified_path(dir_name):
    paths = Path(dir_name).glob('*')
    return max(paths, key=lambda path: path.stat().st_ctime)

In [4]:
leaky_companies = pd.read_csv(last_modified_path(INDEED_RATINGS_DIR))
companies = leaky_companies.drop('indeed_rating', axis=1).dropna()
del leaky_companies

In [5]:
fetch_id = str(uuid.uuid4())

In [6]:
def make_pagespeed_request(page):
    return requests.get(
        url=PAGESPEED_BASE_URL,
        params={
            'url': page,
            'key': PAGESPEED_API_KEY
        }
    )

In [7]:
def unit(display_value):
    return re.search('\w+$', display_value)[0]

def verify_units(audits):
    for key in AUDIT_KEYS:
        if key == 'estimated-input-latency':
            if unit(audits[key]['displayValue']) != 'ms':
                raise BaseException('expected to always receive milliseconds')
        else:
            if unit(audits[key]['displayValue']) != 's':
                raise BaseException('expected to always receive seconds')
            

In [8]:
def displayToFloat(displayValue):
    return float(re.search('^\d+\.?\d*', displayValue)[0])

In [9]:
def audits(response):
    audits = response.json()['lighthouseResult']['audits']
    verify_units(audits)
    return {
        audit_key : displayToFloat(audits[audit_key]['displayValue'])
        for audit_key in AUDIT_KEYS
    }

In [10]:
def site_row(url):
    print(url)
    return dict(
        audits(make_pagespeed_request(url)),
        url=url
    )

In [11]:
def store_result(results, result_ind, url):
    results[result_ind] = site_row(url)

In [12]:
trunced_companies = companies[:TRUNCATE_COMPANIES]

urls = list(trunced_companies['url'])
result_len = len(trunced_companies)
threads = []
results = [None] * result_len
for result_ind in range(result_len):
    thread = Thread(
        target=store_result,
        args=(results, result_ind, urls[result_ind])
    )
    threads.append(thread)
    thread.start()
    time.sleep(0.2)

for thread in threads:
    thread.join()

# results = [
#     site_row(url)
#     for url in trunced_companies['url']
# ]

http://www.abbott.com
https://www.abbvie.com
http://www.abiomed.com
http://www.accenture.com/
http://www.adobe.com
http://www.amd.com/
http://www.advanceautoparts.com/
http://www.aes.com/home/default.aspx
http://www.aflac.com
http://careers.agilent.com/
https://www.instagram.com/akamaicareers
http://alaskaair.jobs
http://www.albemarle.org
http://www.alexionpharm.com
http://www.allegion.com/corp/en/home/contact-us.html
http://www.allergan.com
http://www.alliancedata.com/
http://www.allstate.jobs/
http://www.altria.com/Pages/default.aspx
http://amazon.jobs/
http://www.ameren.com
http://jobs.aa.com


Exception in thread Thread-7:
Traceback (most recent call last):
  File "/Users/lu/.pyenv/versions/3.7.0/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/Users/lu/.pyenv/versions/3.7.0/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-11-b2a9a667430c>", line 2, in store_result
    results[result_ind] = site_row(url)
  File "<ipython-input-10-fa60d633b50b>", line 4, in site_row
    audits(make_pagespeed_request(url)),
  File "<ipython-input-9-86189f674be5>", line 2, in audits
    audits = response.json()['lighthouseResult']['audits']
KeyError: 'lighthouseResult'



http://www.aep.com/
http://Careers.americanexpress.com
http://www.aig.com/careers
http://www.americantower.com
http://www.ameriprise.com


Exception in thread Thread-15:
Traceback (most recent call last):
  File "/Users/lu/.pyenv/versions/3.7.0/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/Users/lu/.pyenv/versions/3.7.0/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-11-b2a9a667430c>", line 2, in store_result
    results[result_ind] = site_row(url)
  File "<ipython-input-10-fa60d633b50b>", line 4, in site_row
    audits(make_pagespeed_request(url)),
  File "<ipython-input-9-86189f674be5>", line 2, in audits
    audits = response.json()['lighthouseResult']['audits']
KeyError: 'lighthouseResult'



http://www.amerisourcebergen.com
http://www.amgen.com


...

Exception in thread Thread-254:
Traceback (most recent call last):
  File "/Users/lu/.pyenv/versions/3.7.0/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/Users/lu/.pyenv/versions/3.7.0/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-11-b2a9a667430c>", line 2, in store_result
    results[result_ind] = site_row(url)
  File "<ipython-input-10-fa60d633b50b>", line 4, in site_row
    audits(make_pagespeed_request(url)),
  File "<ipython-input-9-86189f674be5>", line 2, in audits
    audits = response.json()['lighthouseResult']['audits']
KeyError: 'lighthouseResult'



http://www.sempra.com
http://www.sherwin-williams.com/
https://business.simon.com/
http://www.skyworksinc.com/
http://slgreen.com
http://www.snapon.com
http://www.southwest.com/careers
http://www.stanleyblackanddecker.com/
http://www.starbucks.com/career-center
http://www.statestreet.com/
https://www.suntrust.com/
http://www.symantec.com
http://www.syfcareers.com
https://www.instagram.com/t._rowe_price
https://www.take2games.com
http://www.target.com/
http://TE.com/careers
http://www.ti.com
http://www.textron.com
http://adtrk.tw/BUPe
http://www.tiffany.com
http://www.twitter.com
http://www.tractorsupply.com
http://www.transdigm.com/
http://www.travelers.com
https://www.instagram.com/gotripadvisor
https://www.tysonfoods.com
http://www.udr.com/
http://www.ulta.com
http://usbank.com/
https://www.instagram.com/upjobs
https://ad.doubleclick.net/ddm/clk/318572457;147092162;j?https://www.jobs-ups.com//?Howheard=P4889&utm_source=indeed.com&utm_campaign=nationwide&utm_medium=job_aggregator&utm_

Exception in thread Thread-195:
Traceback (most recent call last):
  File "/Users/lu/.pyenv/versions/3.7.0/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/Users/lu/.pyenv/versions/3.7.0/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-11-b2a9a667430c>", line 2, in store_result
    results[result_ind] = site_row(url)
  File "<ipython-input-10-fa60d633b50b>", line 4, in site_row
    audits(make_pagespeed_request(url)),
  File "<ipython-input-9-86189f674be5>", line 2, in audits
    audits = response.json()['lighthouseResult']['audits']
KeyError: 'lighthouseResult'




http://www.wellsfargo.com
http://www.wdc.com
https://www.westrock.com/
http://www.weyerhaeuser.com
http://www.whirlpoolcorp.com
https://www.instagram.com/willistowerswatson
http://www.wynnresorts.com
http://www.xcelenergy.com
http://www.xerox.com
http://www.xilinx.com
http://www.yum.com
https://www.zimmerbiomet.com/
http://www.zionsbancorporation.com
http://www.zoetis.com/


Exception in thread Thread-248:
Traceback (most recent call last):
  File "/Users/lu/.pyenv/versions/3.7.0/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/Users/lu/.pyenv/versions/3.7.0/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-11-b2a9a667430c>", line 2, in store_result
    results[result_ind] = site_row(url)
  File "<ipython-input-10-fa60d633b50b>", line 4, in site_row
    audits(make_pagespeed_request(url)),
  File "<ipython-input-9-86189f674be5>", line 2, in audits
    audits = response.json()['lighthouseResult']['audits']
KeyError: 'lighthouseResult'

Exception in thread Thread-385:
Traceback (most recent call last):
  File "/Users/lu/.pyenv/versions/3.7.0/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/Users/lu/.pyenv/versions/3.7.0/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-

In [17]:
results = [r for r in results if r is not None]

In [18]:
with open(f'{PAGESPEED_RESULTS_DIR}/{fetch_id}', 'w') as results_file:
    csv_writer = csv.writer(results_file)
    csv_writer.writerow(RESULT_KEYS)
    csv_writer.writerows([
        [result[key] for key in RESULT_KEYS]
        for result in results
    ])

In [19]:
with open(LOG_FILENAME, 'a') as log:
    csv_writer = csv.writer(log)
    csv_writer.writerow([fetch_id, datetime.datetime.now()])